# Projeto 1 - Ciência dos Dados

Nome: Eric Possato

Nome: Gabriel Nichols

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\erica\Desktop\Insper\Sem 2\Ciência de Dados\CDados-Projeto-1


- Fazer mais uma stringassa com as irrevelantes
- Transformar string em series
- Value counts nessas series
- Soma strings para uma stringassazassa
- Transformar em uma MEGASERIES

- Calcular probabilidade de aparecer palavras relevantes len(series)/len(outraseries)
- Calcular probabilidade de uma palavra estar dentro de um conjunto
- Aplicar suavização de laplace
- Aplicar função nos tweets
- Aplicar função na base de testes
- Observar porcentagem de acertos na classificação

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'vaper.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(10)

,Treinamento,Relevancia
0,eu dando teto preto depois de 01 coisada no va...,0
1,minha vó fumando vaper kkkkkkkkkkkkkkkkkkkk,0
2,e eu que ganhei um vaper do meu patrão kkkkkk,0
3,lembrei agora que fiquei bebada segunda e diss...,0
4,aí q vontade de comprar um vaper novo,0
5,aparentemente eu tentei comprar um vaper bebad...,0
6,pensando em comprar um vaper vei,0
7,@negotalles depois que meu amigo tudo começou ...,0
8,minha proxima aquisição vai ser um vaper 🤓,0
9,"onde eu compro vaper aqui no rio de janeiro, q...",0


In [5]:
train['Treinamento'] = train['Treinamento'].apply(lambda x: re.split('https://.*', str(x))[0])
pattern1 = r'(http://[^"\s]+)|(@\w+)'
pattern2 = r'(?<=^|(?<=[^a-zA-Z0-9-_.]))@([A-Za-z]+[A-Za-z0-9]+)'
train['Treinamento'] = train['Treinamento'].str.replace(pattern1, "")
train['Treinamento'] = train['Treinamento'].str.replace(pattern2, "")
train['Treinamento'] = train['Treinamento'].str.replace('[!-.:?;,\/]', '')
train.head(10)

,Treinamento,Relevancia
0,eu dando teto preto depois de 01 coisada no va...,0
1,minha vó fumando vaper kkkkkkkkkkkkkkkkkkkk,0
2,e eu que ganhei um vaper do meu patrão kkkkkk,0
3,lembrei agora que fiquei bebada segunda e diss...,0
4,aí q vontade de comprar um vaper novo,0
5,aparentemente eu tentei comprar um vaper bebad...,0
6,pensando em comprar um vaper vei,0
7,depois que meu amigo tudo começou fumar vaper...,0
8,minha proxima aquisição vai ser um vaper 🤓,0
9,onde eu compro vaper aqui no rio de janeiro qu...,0


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,"viciar en vaper ? serio ? nao to acreditando, ...",0
1,to há meses com um vaper que não acaba!!!!! to...,1
2,"em minha casinha, fumando meu vaper e na paz d...",0
3,@_biagrazyy ela usou um vaper e ainda me mostr...,1
4,@irafaelatavares eu sábado caindo bebeda.\nesp...,0


In [7]:
test['Teste'] = test['Teste'].apply(lambda x: re.split('https://.*', str(x))[0])
pattern1 = r'(http://[^"\s]+)|(@\w+)'
pattern2 = r'(?<=^|(?<=[^a-zA-Z0-9-_.]))@([A-Za-z]+[A-Za-z0-9]+)'
test['Teste'] = test['Teste'].str.replace(pattern1, "")
test['Teste'] = test['Teste'].str.replace(pattern2, "")
test['Teste'] = test['Teste'].str.replace('[!-.:?;,\/]', '')

test

,Teste,Relevancia
0,viciar en vaper serio nao to acreditando me ...,0
1,to há meses com um vaper que não acaba to chocada,1
2,em minha casinha fumando meu vaper e na paz do...,0
3,ela usou um vaper e ainda me mostrou o vídeo ...,1
4,eu sábado caindo bebeda\nespecialidade quebra...,0
...,...,...
195,e se eu comprar um vaper,0
196,eu guardo o vaper tão bem que as vezes vou pro...,0
197,pora só queria meu vaper de novo,0
198,eu falo o vaper em si nunca usei,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Foram considerados relevantes os tweets que: 
    comparavam o produto com outros similarer;
    mencionavem os motivos do usuaria gostar ou não do produto;
    julgaram o produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
train_relevante = train.loc[train['Relevancia']==1,['Treinamento']]

train_relevante.head()

,Treinamento
12,compra um pod vaper é horrível
14,não tá dando condições nem de comprar um vaper...
51,assistindo filme fumando vaper comendo brigade...
52,vaper n tem nada demais ms smp q eu to saindo ...
53,caralho deixei meu rim pra comprar o negócio d...


In [9]:
train_irrelevante = train.loc[train['Relevancia']==0,['Treinamento']]

In [10]:
def prepara_series(frame):
    lista = frame.tolist()
    string = string_relevante = ','.join(lista)
    string = string.replace('\n', ' ')
    string = string.replace(',', ' ')
    string = string.split()
    series = pd.Series(string)
    return series


In [11]:
#criando series com palavras
series_relevante = prepara_series(train_relevante['Treinamento'])
series_irrelevante = prepara_series(train_irrelevante['Treinamento'])
series_total = prepara_series(train['Treinamento'])

In [14]:
#tabelas de frequencia relativa de cada palavra
tabela_relevante_relativa = series_relevante.value_counts(True)
tabela_irrelevante_relativa = series_irrelevante.value_counts(True)
tabela_total_relativa = series_total.value_counts(True)

In [25]:
#probabilidades de palavra ser relevante
prob_relevante = len(series_relevante)/len(series_total)
prob_irrelevante = len(series_irrelevante)/len(series_total)

palavras_sem_repeticao = []
for palavra in series_total:
    if palavra not in palavras_sem_repeticao:
        palavras_sem_repeticao.append(palavra)

In [89]:
def prob_palavra_em_conjunto(palavra):
    if palavra in tabela_relevante_relativa:
        prob_palavra_relevante = tabela_relevante_relativa[palavra]
    else:
        prob_palavra_relevante = 0
    if palavra in tabela_irrelevante_relativa:
        prob_palavra_irrelevante = tabela_irrelevante_relativa[palavra]
    else:
        prob_palavra_irrelevante = 0
           
    return [prob_palavra_relevante,prob_palavra_irrelevante]
def classifica_tweet(frase):
    frase = frase.split()
    prob_frase_relevante = 1
    prob_frase_irrelevante = 1
    
    

    for palavra in frase:
        prob_relevante = prob_palavra_em_conjunto(palavra)[0]
        prob_irrelevante = prob_palavra_em_conjunto(palavra)[1]
    
        relevante_com_laplace = (prob_relevante + 1)/(len(series_relevante)+len(palavras_sem_repeticao))
        irrelevante_com_laplace = (prob_irrelevante + 1)/(len(series_irrelevante)+len(palavras_sem_repeticao))
        
        
        prob_frase_relevante *= relevante_com_laplace
        prob_frase_irrelevante *= irrelevante_com_laplace
    
    if prob_frase_relevante*prob_relevante > prob_frase_irrelevante*prob_irrelevante:
        return 1
    else:
        return 0

0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [95]:
test['Classificado'] = test['Teste'].apply(classifica_tweet)

In [96]:
test

,Teste,Relevancia,Classificado
0,viciar en vaper serio nao to acreditando me ...,0,0
1,to há meses com um vaper que não acaba to chocada,1,0
2,em minha casinha fumando meu vaper e na paz do...,0,0
3,ela usou um vaper e ainda me mostrou o vídeo ...,1,0
4,eu sábado caindo bebeda\nespecialidade quebra...,0,1
...,...,...,...
195,e se eu comprar um vaper,0,1
196,eu guardo o vaper tão bem que as vezes vou pro...,0,0
197,pora só queria meu vaper de novo,0,0
198,eu falo o vaper em si nunca usei,0,0


In [113]:
acuracia = test.loc[(test['Relevancia']==test['Classificado']),:]

print(f'Acurácia: {(len(acuracia)/len(test)*100)}%')


verdadeiro_positivo = test.loc[((test['Relevancia'] == 1) & (test['Classificado'] == 1)),:]
falso_positivo = test.loc[((test['Relevancia'] == 0) & (test['Classificado'] == 1)),:]
verdadeiro_negativo = test.loc[((test['Relevancia'] == 0) & (test['Classificado'] == 0)),:]
falso_negativo = test.loc[((test['Relevancia'] == 1) & (test['Classificado'] == 0)),:]

vp = len(verdadeiro_positivo)/len(test)*100
fp = len(falso_positivo)/len(test)*100
vn = len(verdadeiro_negativo)/len(test)*100
fn = len(falso_negativo)/len(test)*100

print(f'Verdadeiro Positivo: {vp}%')
print(f'Falso Positivo: {fp}%')
print(f'Verdadeiro Negativo: {vn}%')
print(f'Falso Negativo: {fn}%')


Acurácia: 61.5%
Verdadeiro Positivo: 7.000000000000001%
Falso Positivo: 23.0%
Verdadeiro Negativo: 54.50000000000001%
Falso Negativo: 15.5%


75.0%


___
### Concluindo

Analisando os resultados gerados pelo classificador, notamos uma grande quantidade de verdadeiros negativos, assim como uma pequena quantidade de verdadeiros positivos. Isto pode ser explicado com o fato da classificação realizada pelo grupo avaliar um número maior de mensagens como irrelevantes do que como relevantes. Desta forma, o programa classifica mais comumente, frases como irrelevantes.

Como o programa dependeu da classificação inicial do grupo e somente uma quantidade mínima dos tweets incluíam críticas reais sobre o produto, algumas das palavras presentes nas mensagens foram similares nas duas categorias, tanto relevantes, quanto irrelevantes. Isto influenciou na acurácia do classificador.

Como o classificador avalia de acordo com as palavras utilizadas, os casos de sarcasmo ou dupla negação não são considerados no processo de avaliar o tweet, sendo impossível detectar mensagens que avaliam o produto negativamente de maneira sarcástica.

Caso a base de dados de treinamento fosse expandida e fossem coletados mais tweets com críticas óbvias do produto, a precisão do classificador aumentaria drasticamente e poderia se tornar capaz de determinar as opiniões sobre o produto on-line, providenciando a empresa com novas informações úteis na venda do produto.


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**